**Введение**

1. Предсказание фильмов/игр/музыки/видео на основе данных о пользователе  Ассоциация
2. Предсказание и распознавание болезни у пациента  Классификация
3. Предсказание выплаты/невыплаты задолженности     Классификация
4. Прогноз погоды                                   Регрессия
5. Автопилот                                        Классификация

multiclass - один пример имеет один класс
multilabel - один пример может иметь от одного до всех возможных классов

Да. Да, можно - разным диапазонам цены присвоить номера классов. Так это будет задача классификации.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_json('../datasets/train.json')
rows, cols = df.shape
print(f'{rows} rows and {cols} columns')

In [ ]:
df.head()
print('price')

In [ ]:
df.info()

Пустых столбцов нет.

In [ ]:
df.describe()

Есть выбросы по сан. узлам и спальням, так как max значительно больше 75%.

In [ ]:
df.corr(numeric_only=True)

Значения коррелируют как положительно, так и отрицательно.

In [ ]:
df = df[['bathrooms', 'bedrooms', 'interest_level', 'price']]

In [ ]:
plt.figure(figsize=(10,6))
filtered_df = df[df['price'] < 10000] # фильтр чтобы гистограмма не отображала выбросы
sns.histplot(filtered_df['price'], bins=50, kde=True)

plt.xlim(0, 10000)
plt.title('Распределение цен на квартиры')
plt.xlabel('Цена')
plt.ylabel('Количество квартир')
plt.show()


In [ ]:
plt.figure(figsize=(12, 4))

filtered_df = df[df['price'] < 10000]
sns.boxplot(x=filtered_df['price'])

plt.title('boxplot()')
plt.xlabel('цена')

plt.show()

Есть много выбросов, так как цены выходят за границы диапазона Q3 + 1.5*IQR, большая часть цен находится в диапазоне от 2500 до 4000, медианная цена около 3000.

In [ ]:
lower = df['price'].quantile(0.01)
upper = df['price'].quantile(0.99)
new_df = df[(df['price'] > lower) & (df['price'] < upper)]

plt.figure(figsize=(10,6))
sns.histplot(new_df['price'], bins=50, kde=True)

plt.xlim(0, 10000)
plt.title('Распределение цен на квартиры')
plt.xlabel('Цена')
plt.ylabel('Количество квартир')
plt.show()

Часть графика с низкими ценами больше не отображается, потому что квартиры с ценой до ~1600 попали в первый процентиль и были удалены при фильтрации выбросов.

In [ ]:
df['interest_level'].dtype

Значения в interest_level хранятся как строки.

In [ ]:
df['interest_level']

In [ ]:
df['interest_level'].value_counts()

In [ ]:
mapping = {'low': 0, 'medium': 1, 'high': 2}
df['interest_level'] = df['interest_level'].map(mapping)
df.head()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(df['bedrooms'], bins=20)

plt.title('Распределение по количеству спален')
plt.ylabel('Количество квартир')
plt.xlabel('Количество спален')
plt.show()

Выбросов как таковых нет, просто квартир с количеством спален >5 очень мало.

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(df['bathrooms'], bins=10)

plt.title('Распределение по количеству ванных комнат')
plt.ylabel('Количество квартир')
plt.xlabel('Количество ванных комнат')
plt.show()

Ситуация с выбросами та же что со спальнями.

In [ ]:
corr_matrix = df.corr()
print(corr_matrix)

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5, square=True)

Корреляция между признаками и целевым объектом есть, самое большое значение у количества ванных комнат. Наиболее высокая отрицательная зависимость у уровня интереса.

In [ ]:
features = ['bathrooms', 'bedrooms', 'interest_level']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, feature in enumerate(features):
    sns.scatterplot(
        data=corr_matrix,
        x='price',
        y=feature,
        ax=axes[i]
    )
    axes[i].set_title(f'Price vs {feature}')
    axes[i].set_xlabel('Price')
    axes[i].set_ylabel(feature)

plt.tight_layout()
plt.show()

In [ ]:
df['bathrooms_squared'] = df['bathrooms'] ** 2
df['bedrooms_squared'] = df['bedrooms'] ** 2
df['interest_level_squared'] = df['interest_level'] ** 2

In [ ]:
corr_matrix_new = df.corr()
corr_matrix_new

bathroooms_squared и bedrooms_squared стали коррелировать с ценой больше чем не возведенные в квадрат значения. interest_level_squared имеет меньшую отрицательную корреляцию с ценой по сравнению с interest_level.

In [ ]:
train = pd.read_json('../datasets/train.json')
test = pd.read_json('../datasets/test.json')

X_train = train.select_dtypes(include=['number']).drop('price', axis=1)
X_test = test.select_dtypes(include=['number']).drop('price', axis=1)

y_train = train['price']
y_test = test['price']

In [ ]:
X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(X_test)

In [ ]:
poly = PolynomialFeatures(degree=10, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

In [ ]:
result_MAE = pd.DataFrame(columns=['model', 'train', 'test'])
result_RMSE = pd.DataFrame(columns=['model', 'train', 'test'])

In [ ]:
lr = LinearRegression()
lr.fit(X_train_poly, y_train)

train_preds = lr.predict(X_train_poly)
test_preds = lr.predict(X_test_poly)

train['lr_prediction'] = train_preds
test['lr_prediction'] = test_preds

In [ ]:
mae_train = mean_absolute_error(y_train, train_preds)
mae_test = mean_absolute_error(y_test, test_preds)

rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))

print(f'MAE train: {mae_train:.2f}')
print(f'MAE test: {mae_test:.2f}')
print(f'RMSE test: {rmse_test:.2f}')
print(f'RMSE train: {rmse_train:.2f}')

In [ ]:
result_MAE.loc[len(result_MAE)] = ['linear_regression', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['linear_regression', rmse_train, rmse_test]

In [ ]:
dtree = DecisionTreeRegressor(random_state=21)

param_grid = {
    "max_depth": [3, 5, 7, 9],
    "min_samples_leaf": [1, 3, 5],
    "max_features": [None, "sqrt", "log2"],
    "criterion": ["mse", "friedman_mse"]
}

grid_search = GridSearchCV(dtree, param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search.fit(X_train_poly, y_train)

best_params = grid_search.best_params_

print(f'Best params = {best_params}')

In [ ]:
best_dtree = DecisionTreeRegressor(**best_params, random_state=21)
best_dtree.fit(X_train_poly, y_train)
dtree_train_pred = best_dtree.predict(X_train_poly)
dtree_test_pred = best_dtree.predict(X_test_poly)

train['dtree_prediction'] = dtree_train_pred
test['dtree_prediction'] = dtree_test_pred

In [ ]:
mae_train = mean_absolute_error(y_train, dtree_train_pred)
mae_test = mean_absolute_error(y_test, dtree_test_pred)

rmse_train = np.sqrt(mean_squared_error(y_train, dtree_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, dtree_test_pred))

print(f'MAE train: {mae_train:.2f}')
print(f'MAE test: {mae_test:.2f}')
print(f'RMSE test: {rmse_test:.2f}')
print(f'RMSE train: {rmse_train:.2f}')

In [ ]:
result_MAE.loc[len(result_MAE)] = ['decision_tree', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['decision_tree', rmse_train, rmse_test]

In [ ]:
mean_price_train = y_train.mean()
mean_price_test = y_test.mean()

median_price_train = y_train.median()
median_price_test = y_test.median()

train['naive_prediction_mean'] = mean_price_train
train['naive_prediction_median'] = median_price_train

test['naive_prediction_mean'] = mean_price_test
test['naive_prediction_median'] = median_price_test

In [ ]:
mae_train_mean = mean_absolute_error(y_train, np.full_like(y_train, mean_price_train))
mae_test_mean = mean_absolute_error(y_test, np.full_like(y_test, mean_price_train))

mae_train_median = mean_absolute_error(y_train, np.full_like(y_train, median_price_train))
mae_test_median = mean_absolute_error(y_test, np.full_like(y_test, median_price_train))

rmse_train_mean = np.sqrt(mean_squared_error(y_train, np.full_like(y_train, mean_price_train)))
rmse_test_mean = np.sqrt(mean_squared_error(y_test, np.full_like(y_test, mean_price_train)))

rmse_train_median = np.sqrt(mean_squared_error(y_train, np.full_like(y_train, median_price_train)))
rmse_test_median = np.sqrt(mean_squared_error(y_test, np.full_like(y_test, median_price_train)))

In [ ]:
result_MAE.loc[len(result_MAE)] = ['naive_mean', mae_train_mean, mae_test_mean]
result_MAE.loc[len(result_MAE)] = ['naive_median', mae_train_median, mae_test_median]

result_RMSE.loc[len(result_RMSE)] = ['naive_mean', rmse_train_mean, rmse_test_mean]
result_RMSE.loc[len(result_RMSE)] = ['naive_median', rmse_train_median, rmse_test_median]

In [ ]:
print(result_MAE)

In [ ]:
print(result_RMSE)

Лучшей моделью является DecisionTreeRegressor, так как у нее наименьшие показатели ошибок.